# Create splits
```
input: trialdata.csv
output: splits folder
```
Trialdata.csv is created by psiturk when you download the datafile. This is the experiment data collected by the task stored in ```Experiment code```

The splits folder will contain a named folder per subject. Test data is filtered out by inspecting the user id. Incomplete data is filtered out by inspecting the number of finished games. These folders contain data that can be fitted to a model by running ```Model code/matlab wrapper/auto_fit.sh``` on it. After model fitting is complete you can create paramsMatrix further down in this notebook.

In [1]:
input_file = "../data/trialdata.csv"
output_folder = "../data"

In [2]:
from fourinarowfunctions import *

data_dict = load_data(input_file)

drop test_im:R_2YspUds9g1Pyp6J
drop test_im:R_1LYfkmETpZRD3kn
drop debug:R_3G7Yq6aginiNva2
drop test_im2:R_2YspUds9g1Pyp6J
drop im_full_test:assign_full_test
drop test_im3:R_2YspUds9g1Pyp6J
drop null:R_emsNDLsvWMLPb7X
drop test_im_ff:test_FF_win
drop test_im_edge:test_edge_win
drop twindemo:noas
drop null:R_2qCyb2CeLwcRxH2
drop null:R_25HjvhOMhc79iaa
drop debugYZNI1G:debugDVJFSO
drop test_BO_11:R_1eXnot6X58rnPMv
drop null:R_2zhw77FMGEb36zK
drop debugOCOGDF:debug2TDLTY
drop test_im_ff2:test_FF2_win
drop null:R_2aV9fwk4CR3GjBg
drop debugQWOQ1C:debugOQ8IEY
drop CPTest:R_1QxPuBVlzMbldXa
drop CPTest:R_1FsoM2AkWpLn1Qw
drop im_full_test:ssign_full_test
drop just_a_test_:test
drop debugfull:noas
drop null:R_vou3qrtMy6aHysp
drop debugscreenshot:noas
Loaded 192 participants


In [3]:
import os
import numpy as np

#helper functions that encode boards as and moves as integers for the model fitting pipeline
def encode_board(pieces):
    return np.sum([2**i for i,p in enumerate(pieces) if p=='1']).astype(np.int64)

def ensure_dir(dirname):
    if not os.path.exists(dirname):
        os.mkdir(dirname)

def save_raw_splits(data_dict, root):
    ensure_dir(root + '/raw/')
    ensure_dir(root + '/splits/')
    numOk = 0
    for username, data in data_dict.items():
        print(f"writing {username} {len(data)}", end='\r')
        try:
            parsed_data = get_parsed_data(data)
        except AssertionError as e:
            print(f"Incomplete {username}: {e}")
            continue
        filename = username.replace(":", "-")
        with open(root + '/raw/' + filename + '.csv', 'w') as f:
            df = pd.DataFrame([(encode_board(bp), encode_board(wp), c.lower().capitalize(), 2**m, rt, username.split(':')[0]) for bp,wp,m,c,rt in parsed_data])
            f.write(df.to_csv(None, index=False, header=False, sep='\t', line_terminator ='\n')[:-1])
        group = (5 * (np.random.permutation(len(parsed_data))/len(parsed_data))).astype(int) + 1
        ensure_dir(root + '/splits/' + filename)
        with open(root + '/splits/' + filename + '/data.csv', 'w') as f:
            df = pd.DataFrame([(encode_board(bp),encode_board(wp), c.lower().capitalize(), 2**m, rt,g,username.split(':')[0]) for (bp,wp,m,c,rt), g in zip(parsed_data, group)])
            f.write(df.to_csv(None, index = False, header=False, sep='\t', line_terminator='\n')[:-1])
        for g in range(1,6):
            with open(root + '/splits/' + filename + '/' + str(g) + '.csv', 'w') as f:
                df = pd.DataFrame([(encode_board(bp), encode_board(wp), c.lower().capitalize(), 2**m, rt,g,username.split(':')[0]) for (bp,wp,m,c,rt), g in zip(parsed_data, group)])
                f.write(df[df[5]==g].to_csv(None, index=False, header=False, sep='\t', line_terminator='\n')[:-1])
        numOk += 1
    print(f"Wrote splits for {numOk} participants{' ' * 20}")

save_raw_splits(data_dict, output_folder)

Incomplete A108:R_2TnOx7dVPSonpTg: user only finished 27 games
Incomplete A123:R_2viEesh2c9XCBMe: user only finished 27 games
Incomplete A120:R_PONVLZnwDK7wSpr: user only finished 27 games
Incomplete 115:R_241tt6O6tVQs9NR: user only finished 22 games
Incomplete 106:R_1HqNZ7zU4Tb0SSR: user only finished 22 games
Incomplete A1174:R_296zFssAEmp2lpK: user only finished 17 games
Incomplete 131:R_3iJ8A5hXnygbqAF: user only finished 22 games
Incomplete 160:R_3h5POosYswsLZqe: user only finished 22 games
Incomplete 159:R_3kolXJjcfDsjL6k: user only finished 22 games
Incomplete A125:R_d5dJPkPEZBF0HLP: user only finished 32 games
Incomplete 128:R_AEYAh7gikx2YEvL: user only finished 22 games
Incomplete 154:R_3kgZuiVj87g1GNO: user only finished 22 games
Incomplete 153:R_reR4dauvBGinQvT: user only finished 22 games
Incomplete 109:R_1NurOLY6ru1XwrL: user only finished 12 games
Incomplete 124:R_cOwNfKZ17YDGcGR: user only finished 12 games
Incomplete 130:R_10pkDu4x8ARiMHf: user only finished 12 games
In

# Create paramsMatrix
```
input: fitted parameters
output: paramsMatrix.csv
```
The input structure is a root folder that contains folders with fitted parameters, like this:
```
data folder
 +- subject_id_1
 |  +- params1.csv
 |  +- params2.csv
 |  +- ...
 +- subject_id_2
 |  +- params1.csv
 ...
```
Use paramsMatrix.csv in the ```Calculate metrics and Elo``` notebook.

In [4]:
data_folder = "../data/splits"
output_file = "../data/paramsMatrix.csv"

import os
import re

parampattern = "params(\\d+).csv"

lines = []
num_subjects = 0
for subject_id in os.listdir(data_folder):
    subdir = os.path.join(data_folder, subject_id)
    if not os.path.isdir(subdir):
        # We're looking for directories only. Skip everything else
        continue
    num_subjects += 1
    for filename in os.listdir(subdir):
        filepath = os.path.join(subdir, filename)
        match = re.match(parampattern, filename)
        if not os.path.isfile(filepath) or not match:
            # Skip any file that isn't a param*.csv file
            continue
        fold_number = match.group(1)
        with open(filepath) as infile:
            lines.append([subject_id.replace("-",":"), int(fold_number), infile.readline().strip()])
lines.sort()
linecounter = 0
with open(output_file, "w") as targetfile:
    # Write the header
    targetfile.write(",subject,fold,")
    targetfile.write(",".join([str(item) for item in range(1,11)]))
    targetfile.write("\n")
    # Write content
    for line in lines:
        line.insert(0, linecounter)
        targetfile.write(",".join([str(item) for item in line]))
        targetfile.write("\n")
        linecounter += 1
print(f"Found {num_subjects} subjects. Wrote {len(lines)} lines to {output_file}")

Found 158 subjects. Wrote 790 lines to ../data/paramsMatrix.csv
